In [ ]:
%load_ext autoreload
%autoreload 2

from model import RetrieverModel
from datasets import load_dataset

# loss = "siglip"
loss = "clip"
batch_size = 32
# epochs = 2 * batch_size
epochs = 1
lr = 1e-7
eval_batch = 1000

model = RetrieverModel(loss_type=loss)
dataset = load_dataset("microsoft/ms_marco", "v2.1")

lr_n = "" if lr == 1e-7 else f"lr{lr:.0E}_"
b_n = "" if batch_size == 2 else f"b{batch_size}"

model_name = model.model_name.split("/")[-1]
model_path = f"{loss}/{model_name}/{b_n + lr_n}"
print(model_path)
print(dataset)

c:\Users\Simon\miniconda3\envs\MSE\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
from transformers import Trainer, TrainingArguments
from callbacks import NotebookProgressCallbackNoTable
from transformers.utils.notebook import NotebookProgressCallback
from evaluation import compute_metrics
import numpy as np

os.environ["WANDB_PROJECT"] = "MSE"
os.environ["WANDB_LOG_MODEL"] = "false"

training_args = TrainingArguments(
    output_dir="models/" + model_path,
    per_device_train_batch_size=batch_size,
    num_train_epochs=epochs,
    learning_rate=lr,
    save_steps=1000,
    save_total_limit=1,
    remove_unused_columns=False,
    bf16=True,
    optim="adamw_bnb_8bit",
    logging_steps=100,
    eval_steps=200,
    eval_strategy="steps",
    eval_on_start=True,
    per_device_eval_batch_size=eval_batch,
    run_name=model_path,
    report_to='wandb',
    # max_steps=1000,
)

def collate_fn(batch):
    def get_relevant(passages):
        passage = np.array(passages["passage_text"])[np.array(passages["is_selected"], bool)]
        return passage[0] if len(passage) > 0 else ""
    res = {
        "query": [ex["query"] for ex in batch],
        # "answer": [ex["answers"][0] if ex["answers"] else "" for ex in batch],
        "answer": [get_relevant(ex["passages"]) for ex in batch],
    }
    return res

trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

trainer.remove_callback(NotebookProgressCallback)
trainer.add_callback(NotebookProgressCallbackNoTable)

# trainer.evaluate()
trainer.train()
# try: trainer.train(resume_from_checkpoint=True)
# except: trainer.train(resume_from_checkpoint=False)

wandb: Currently logged in as: simon-doehl (simon-doehl-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


TrainOutput(global_step=25730, training_loss=3.094243464078877, metrics={'train_runtime': 2835.6683, 'train_samples_per_second': 290.323, 'train_steps_per_second': 9.074, 'total_flos': 0.0, 'train_loss': 3.094243464078877, 'epoch': 10.0})